# This is a Jupyter Notebook

In [1]:
# Import scikit-plot
import scikitplot as sp
from scikitplot import stats

# Enable notebook info logger mode
logger = sp.sp_logging.setup_logging_for_jupyter()
sp.__version__

2025-05-29 12:51:26.825813: D scikitplot 132020496942144 _citation.py:61:_get_bibtex] _bibtex from attr CITATION text
2025-05-29 12:51:29.950477: D scikitplot 132020496942144 165456975.py:6:<module>] Falling back to logging.setup_logging_for_jupyter


TypeError: 'NoneType' object is not callable

In [ ]:
# Set up logging
# import logging
# logging.basicConfig(level=logging.DEBUG)
# logger = logging.getLogger(__name__)

In [ ]:
import os
import time
from pprint import pprint

import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
import seaborn as sns